In [1]:
import os
import shutil
import importlib
import copy
import glob
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
import torch
import torchaudio
from tqdm import tqdm  # progress bar
import phaselocknet_model
import util

importlib.reload(phaselocknet_model)
importlib.reload(util)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [2]:
regex_filenames = "/media/marmoset/data/code_package/phaselock/localization/speech_in_noise_in_reverb_stim.hdf5"
sr = 50000 if "localization" in regex_filenames else 20000
sr_src = 44100
dataset = util.HDF5Dataset(regex_filenames)

In [21]:
dir_model = "../phaselocknet_torch/models/sound_localization/simplified_IHC3000/arch09"
# dir_model = "../phaselocknet_torch/models/sound_localization/simplified_IHC0050/arch01"
model, config_model = phaselocknet_model.get_model(
    dir_model=dir_model,
    fn_config="config.json",
    fn_arch="arch.json",
)
# Load model weights from torch instead of tensorflow checkpoint
util.load_model_checkpoint(
    model=model.perceptual_model,
    dir_model=dir_model,
    fn_ckpt="ckpt_BEST.pt",
    weights_only=True,
)
model.train(mode=False)
model.to(device)
assert not model.training

[get_model] dir_model='../phaselocknet_torch/models/sound_localization/simplified_IHC3000/arch09'
[get_model] |__ input_shape=[2, 60000, 2]
[get_model] |__ config_random_slice={'size': [50, 10000], 'buffer': [0, 1000]}
[load_model_checkpoint] ../phaselocknet_torch/models/sound_localization/simplified_IHC3000/arch09/ckpt_BEST.pt


In [22]:
model

Model(
  (peripheral_model): PeripheralModel(
    (body): Sequential(
      (cochlear_filterbank): GammatoneFilterbank(
        (fb): FIRFilterbank()
      )
      (ihc_transduction): IHCTransduction()
      (ihc_lowpass_filter): IHCLowpassFilter()
      (anf_rate_level): SigmoidRateLevelFunction(
        (envelope_function): HilbertEnvelope(
          (hilbert): Hilbert()
        )
      )
      (anf_spike_generator): BinomialSpikeGenerator()
    )
    (head): RandomSlice(
      (crop): RandomCrop(size=(50, 10000), padding=None)
    )
  )
  (perceptual_model): PerceptualModel(
    (body): Sequential(
      (conv_0): CustomPaddedConv2d(6, 32, kernel_size=(3, 32), stride=(1, 1))
      (pool_0): MaxPool2d(kernel_size=[1, 1], stride=[1, 1], padding=0, dilation=1, ceil_mode=False)
      (relu_0): ReLU()
      (batch_norm_0): SyncBatchNorm(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv_1): CustomPaddedConv2d(32, 32, kernel_size=(2, 16), stride=(1, 1))
      

In [23]:
resampler = torchaudio.transforms.Resample(orig_freq=sr_src, new_freq=sr)
tap_name = "perceptual_model.body.fc_intermediate"
# tap_name = "perceptual_model.head.label_loc_int.fc_output"
handle = model.get_submodule(tap_name)
features = {}
def hook(_m, _in, output):
    features[tap_name] = output.detach()      
hook_handle = handle.register_forward_hook(hook)
model.eval()

all_feats, all_azim, all_index_room, all_label_word, all_index_stim, all_snr = [], [], [], [], [], []
with torch.inference_mode():
    for i in tqdm(range(len(dataset))):
        sample = dataset[i]  # dict per sample
        x_raw = torch.tensor(sample["signal"], dtype=torch.float32)[None, ...] # (1, T, C)
        x_pre = torch.stack([resampler(x_raw[..., ch]) for ch in range(x_raw.shape[-1])], dim=-1)
        x = util.pad_or_trim_to_len(x_pre, n=model.input_shape[1], dim=1)
        features.clear()
        _ = model(x.to(device))
        feat = features[tap_name].detach().squeeze(0).cpu().numpy()  # torch.Size([1, 512]) to array(512,)
        all_feats.append(feat)
        all_azim.append(sample["foreground_azim"]) # 0 to 375 smooth change
        all_index_room.append(sample["index_room"]) # 0 1 2 3 4
        all_label_word.append(sample["label_word_int"]) # 376 words in speech
        all_index_stim.append(sample["index"])
        all_snr.append(sample["snr"]) # 9 SNRs and 1 inf

hook_handle.remove()
all_feats_np = np.stack(all_feats, axis=0)  # (N, 512)
np.save("reverb_all_emb_penultimate_512.npy", all_feats_np)
# np.save("reverb_all_azim.npy", np.array(all_azim))
# np.save("reverb_all_index_room.npy", np.array(all_index_room))
# np.save("reverb_all_label_word.npy", np.array(all_label_word))
# np.save("reverb_all_index_stim.npy", np.array(all_index_stim))
# np.save("reverb_all_snr.npy", np.array(all_snr))


100%|█████████████████████████████████████| 18800/18800 [08:33<00:00, 36.59it/s]
